## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")


In [17]:
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,North Bend,43.4065,-124.2243,52.86,85,75,17.27,broken clouds,US
1,1,Vaini,-21.2000,-175.2000,82.56,83,75,6.91,broken clouds,TO
2,2,Jamestown,42.0970,-79.2353,54.99,49,0,10.36,clear sky,US
3,3,Komsomolskiy,40.4272,71.7189,51.22,87,100,4.61,mist,UZ
4,4,Makakilo City,21.3469,-158.0858,79.12,53,0,10.36,clear sky,US


In [18]:
city_data_df.shape

(551, 10)

In [52]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
print(f'Mimunum Temperature Selected :{min_temp}')
print(f'Maximum Temperature Selected :{max_temp}')

Mimunum Temperature Selected :30.0
Maximum Temperature Selected :70.0


In [53]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                 (city_data_df["Max Temp"] <= max_temp)]

In [54]:
# 4a. Determine if there are any empty rows.

filtered_city_data.count()

City_ID                221
City                   221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
Country                216
dtype: int64

In [55]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = filtered_city_data.dropna()

In [56]:
clean_df.count()

City_ID                216
City                   216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
Country                216
dtype: int64

In [57]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,North Bend,US,52.86,broken clouds,43.4065,-124.2243,
2,Jamestown,US,54.99,clear sky,42.0970,-79.2353,
3,Komsomolskiy,UZ,51.22,mist,40.4272,71.7189,
5,Bredasdorp,ZA,69.28,broken clouds,-34.5322,20.0403,
9,Shache,CN,41.88,scattered clouds,38.4167,77.2406,
12,Half Moon Bay,US,57.88,haze,37.4636,-122.4286,
13,Albany,US,40.95,overcast clouds,42.6001,-73.9662,
14,Besikduzu,TR,44.49,overcast clouds,41.0520,39.2284,
15,Samfya,ZM,65.41,light rain,-11.3649,29.5565,
17,Neuquen,AR,60.69,clear sky,-38.9516,-68.0591,


In [58]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 

    hotels = requests.get(base_url, params= params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [59]:
# 7. Drop the rows where there is no Hotel Name.
#There is no row to Drop 
hotel_df.isnull().sum()


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [60]:
clean_hotel_df = hotel_df

In [61]:
clean_hotel_df.shape

(216, 7)

In [62]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [66]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name<b></dt><dd>{Hotel Name}</dd>
<dt><b>City<b></dt><dd>{City}</dd>
<dt><b>Country<b></dt><dd>{Country}</dd>
<dt><b>Current Weather<b></dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [68]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))